# Configure Aggregate Module Params

This notebook should be used as a test for ensuring correct aggregate parameters before aggregate processing.
Cells marked with <font color='red'>SET PARAMETERS</font> contain crucial variables that need to be set according to your specific experimental setup and data organization.
Please review and modify these variables as needed before proceeding with the analysis.

## <font color='red'>SET PARAMETERS</font>

### Fixed parameters for aggregate module

- `CONFIG_FILE_PATH`: Path to a Brieflow config file used during processing. Absolute or relative to where workflows are run from.

In [ ]:
CONFIG_FILE_PATH = "config/config.yml"

In [ ]:
from pathlib import Path

import yaml
import pandas as pd
import matplotlib.pyplot as plt

from lib.shared.file_utils import get_filename
from lib.aggregate.load_format_data import clean_cell_data, load_parquet_subset
from lib.aggregate.feature_processing import feature_transform, grouped_standardization
from lib.aggregate.collapse_data import collapse_to_sgrna, collapse_to_gene
from lib.aggregate.eval_aggregate import suggest_parameters
from lib.aggregate.cell_classification import (
    plot_mitotic_distribution_hist,
    plot_mitotic_distribution_scatter,
    split_mitotic_simple,
)
from lib.aggregate.montage_utils import create_cell_montage, add_filenames
from lib.shared.configuration_utils import CONFIG_FILE_HEADER

## <font color='red'>SET PARAMETERS</font>

### Testing on subset of data

- `TEST_PLATE`: Plate used for testing configuration 
- `TEST_WELL`: Well identifier used for testing configuration 
- `POPULATION_FEATURE`: The column name that identifies your perturbation groups (e.g., 'gene_symbol_0' for CRISPR screens, 'treatment' for drug screens)
- `FILTER_SINGLE_GENE`: Whether or not to ONLY keep cells with mapped_single_gene=True.

In [ ]:
TEST_PLATE = None
TEST_WELL = None

POPULATION_FEATURE = None
FILTER_SINGLE_GENE = None

In [ ]:
# load config file and determine root path
with open(CONFIG_FILE_PATH, "r") as config_file:
    config = yaml.safe_load(config_file)
ROOT_FP = Path(config["all"]["root_fp"])

# Load subset of data
# Takes ~1 minute
merge_final_fp = (
    ROOT_FP
    / "merge"
    / "parquets"
    / get_filename({"plate": TEST_PLATE, "well": TEST_WELL}, "merge_final", "parquet")
)
merge_final = load_parquet_subset(merge_final_fp)
print(f"Unique populations: {merge_final[POPULATION_FEATURE].nunique()}")

# Remove unassigned cells
clean_df = clean_cell_data(
    merge_final, POPULATION_FEATURE, filter_single_gene=FILTER_SINGLE_GENE
)
print(f"Loaded {len(merge_final)} cells with {len(merge_final.columns)} features")

## <font color='red'>SET PARAMETERS</font>

### Apply feature transformations

- `TRANFORMATIONS_FP`: CSV file containing feature transformation specifications. Each row defines a feature pattern and its transformation (e.g., 'log(feature)', 'log(feature-1)'), and should have a feature and transformation column

In [ ]:
TRANFORMATIONS_FP = None

In [ ]:
# load lower case version of channels
channels = [channel.lower() for channel in config["phenotype"]["channel_names"]]

# load transformations
transformations = pd.read_csv(TRANFORMATIONS_FP, sep="\t")

# perform feature transformation
transformed_df = feature_transform(clean_df, transformations, channels)
transformed_df

## <font color='red'>SET PARAMETERS</font>

### Standardize features

- `CONTROL_PREFIX`: Prefix identifying control populations.
- `GROUP_COLUMNS`: Columns defining experimental groups (e.g., `['well']` for per-well standardization).
- `INDEX_COLUMNS`: Columns uniquely identifying cells (e.g., `['tile', 'cell_0']`).
- `CAT_COLUMNS`: Categorical columns to preserve.
- `FEATURE_START`: First column containing measured features.

We provide a useful function for suggesting these parameters, `suggest_parameters`.

In [ ]:
suggest_parameters(clean_df, POPULATION_FEATURE)

In [ ]:
CONTROL_PREFIX = None
GROUP_COLUMNS = None
INDEX_COLUMNS = None
CAT_COLUMNS = None
FEATURE_START = None

In [ ]:
# Identify features to standardize (all columns after mapped_single_gene)
feature_start_idx = transformed_df.columns.get_loc(FEATURE_START)
target_features = transformed_df.columns[feature_start_idx:].tolist()
# Standardize the data
standardized_df = grouped_standardization(
    transformed_df,
    population_feature=POPULATION_FEATURE,
    control_prefix=CONTROL_PREFIX,
    group_columns=GROUP_COLUMNS,
    index_columns=INDEX_COLUMNS,
    cat_columns=CAT_COLUMNS,
    target_features=target_features,
    drop_features=False,
)

# add phenotype image filenames for each well/tile
standardized_df = add_filenames(standardized_df, ROOT_FP)

standardized_df

## <font color='red'>SET PARAMETERS</font>

### Split mitotic and interphase data

- `MITOTIC_THRESHOLD_VARIABLE`: Column name used to identify mitotic cells (e.g., 'nucleus_dapi_int' for DAPI intensity)  
- `MITOTIC_THRESHOLD`: Numerical threshold to separate mitotic from interphase cells (examine intensity histogram to determine appropriate value)

**Note**: One can test multiple thresholds below and decide on a final thresholding strategy decided later.

In [ ]:
# thresholding with variable 1
MITOTIC_THRESHOLD_VARIABLE_X = None
MITOTIC_THRESHOLD_X = None

# thresholding with variable 2
MITOTIC_THRESHOLD_VARIABLE_Y = None
MITOTIC_THRESHOLD_Y = None

In [ ]:
percent_mitotic = plot_mitotic_distribution_hist(
    standardized_df,
    threshold_variable=MITOTIC_THRESHOLD_VARIABLE_X,
    threshold_value=MITOTIC_THRESHOLD_X,
)

percent_mitotic = plot_mitotic_distribution_hist(
    standardized_df,
    threshold_variable=MITOTIC_THRESHOLD_VARIABLE_Y,
    threshold_value=MITOTIC_THRESHOLD_Y,
)

plot_mitotic_distribution_scatter(
    standardized_df,
    threshold_variable_x=MITOTIC_THRESHOLD_VARIABLE_X,
    threshold_variable_y=MITOTIC_THRESHOLD_VARIABLE_Y,
    threshold_x=MITOTIC_THRESHOLD_X,
    threshold_y=MITOTIC_THRESHOLD_Y,
    alpha=0.1,
)

## <font color='red'>SET PARAMETERS</font>

### Final mitotic thresholding

- `THRESHOLD_CONDITIONS`: Columns, values, and direction to use for thresholding. Use the montages generated below to assess thresholding strategy.

In [ ]:
# Final thresholding strategy
THRESHOLD_CONDITIONS = None

In [ ]:
# Use final thresholding to split cells
mitotic_df, interphase_df = split_mitotic_simple(standardized_df, THRESHOLD_CONDITIONS)
print(
    f"Subsetting {len(mitotic_df)} mitotic cells and {len(interphase_df)} interphase cells"
)

# Get channels from config
CHANNELS = config["phenotype"]["channel_names"]

# Create figure to evaluate DAPI cutoff

titles = [
    "Randomly Selected Interphase Cells - DAPI",
    "Randomly Selected Mitotic Cells - DAPI",
    "Highest DAPI Median Interphase Cells - DAPI",
    "Lowest DAPI Median Mitotic Cells - DAPI",
]

montages = [
    create_cell_montage(
        cell_data=interphase_df,
        channels=CHANNELS,
        selection_params={
            "method": "random",
        },
    )["DAPI"],
    create_cell_montage(
        cell_data=mitotic_df,
        channels=CHANNELS,
        selection_params={
            "method": "random",
        },
    )["DAPI"],
    create_cell_montage(
        cell_data=interphase_df,
        channels=CHANNELS,
        selection_params={
            "method": "sorted",
            "sort_by": "nucleus_DAPI_median",
            "ascending": False,
        },
    )["DAPI"],
    create_cell_montage(
        cell_data=mitotic_df,
        channels=CHANNELS,
        selection_params={
            "method": "sorted",
            "sort_by": "nucleus_DAPI_median",
            "ascending": True,
        },
    )["DAPI"],
]

# Initialize figure and subplots
fig, axes = plt.subplots(2, 2, figsize=(10, 4))

# Display each montage
for ax, title, montage in zip(axes.flat, titles, montages):
    ax.imshow(montage, cmap="gray")
    ax.set_title(title, fontsize=14)
    ax.axis("off")

# Adjust layout and show the plot
plt.tight_layout()
plt.show()

In [ ]:
# Re-standardize mitotic population
mitotic_standardized_df = grouped_standardization(
    mitotic_df,
    population_feature=POPULATION_FEATURE,
    control_prefix=CONTROL_PREFIX,
    group_columns=GROUP_COLUMNS,
    index_columns=INDEX_COLUMNS,
    cat_columns=CAT_COLUMNS,
    target_features=target_features,
    drop_features=True,
)

# Re-standardize interphase population
interphase_standardized_df = grouped_standardization(
    interphase_df,
    population_feature=POPULATION_FEATURE,
    control_prefix=CONTROL_PREFIX,
    group_columns=GROUP_COLUMNS,
    index_columns=INDEX_COLUMNS,
    cat_columns=CAT_COLUMNS,
    target_features=target_features,
    drop_features=True,
)

# Get sgrna summaries for mitotic
mitotic_sgrna_df = collapse_to_sgrna(
    mitotic_standardized_df,
    method="median",
    target_features=target_features,
    index_features=[POPULATION_FEATURE, "sgRNA_0"],
    control_prefix=CONTROL_PREFIX,
)

# Get sgrna summaries for interphase
interphase_sgrna_df = collapse_to_sgrna(
    interphase_standardized_df,
    method="median",
    target_features=target_features,
    index_features=[POPULATION_FEATURE, "sgRNA_0"],
    control_prefix=CONTROL_PREFIX,
)

# Get gene summaries for mitotic
mitotic_gene_df = collapse_to_gene(
    mitotic_sgrna_df,
    target_features=target_features,
    index_features=[POPULATION_FEATURE],
)

# Get gene summaries for interphase
interphase_gene_df = collapse_to_gene(
    interphase_sgrna_df,
    target_features=target_features,
    index_features=[POPULATION_FEATURE],
)

# Show summary of subset aggregation
summary = pd.DataFrame(
    {
        "Stage": [
            "Raw Data",
            "Mitotic Cells",
            "Interphase Cells",
            "Mitotic sgRNAs",
            "Interphase sgRNAs",
            "Mitotic Genes",
            "Interphase Genes",
        ],
        "Count": [
            len(clean_df),
            len(mitotic_df),
            len(interphase_df),
            len(mitotic_sgrna_df),
            len(interphase_sgrna_df),
            len(mitotic_gene_df),
            len(interphase_gene_df),
        ],
    }
)
print("\nAnalysis Summary of Data Subset:")
summary

## Add aggregate parameters to config file

In [ ]:
# Add aggregate section
config["aggregate"] = {
    "transformations_fp": TRANFORMATIONS_FP,
    "population_feature": POPULATION_FEATURE,
    "filter_single_gene": FILTER_SINGLE_GENE,
    "feature_start": FEATURE_START,
    "control_prefix": CONTROL_PREFIX,
    "group_columns": GROUP_COLUMNS,
    "index_columns": INDEX_COLUMNS,
    "cat_columns": CAT_COLUMNS,
    "threshold_conditions": THRESHOLD_CONDITIONS,
}

# Write the updated configuration
with open(CONFIG_FILE_PATH, "w") as config_file:
    # Write the introductory comments
    config_file.write(CONFIG_FILE_HEADER)

    # Dump the updated YAML structure, keeping markdown comments for sections
    yaml.dump(config, config_file, default_flow_style=False)